In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import joblib

In [ ]:
# Fungsi untuk menghasilkan rekomendasi berdasarkan persentase keuntungan dan pendapatan
def generate_recommendations_v2(pendapatan, keuntungan):
    recommendations = []

    # Keuntungan rendah
    if keuntungan / pendapatan < 0.1:
        recommendations.append("Tawarkan diskon besar untuk menarik pembeli.")
        recommendations.append("Gunakan strategi bundling dengan produk lain.")
        recommendations.append("Pertimbangkan untuk mencari pemasok yang lebih murah.")

    # Keuntungan sedang
    elif 0.1 <= keuntungan / pendapatan < 0.3:
        recommendations.append("Diskon moderat untuk meningkatkan volume penjualan.")
        recommendations.append("Pertimbangkan untuk meningkatkan harga untuk meningkatkan margin.")
        recommendations.append("Lakukan promosi di platform e-commerce dengan potongan harga.")

    # Keuntungan tinggi
    else:
        recommendations.append("Pertahankan harga saat ini karena margin yang tinggi.")
        recommendations.append("Investasikan dalam pemasaran untuk memperluas jangkauan pasar.")
        recommendations.append("Fokus pada strategi penjualan premium untuk margin lebih besar.")

    # Pendapatan rendah
    if pendapatan < 2000000:
        recommendations.append("Berikan diskon 10% untuk menarik pelanggan baru.")

    # Pendapatan sedang
    if 2000000 <= pendapatan < 5000000:
        recommendations.append("Coba promosi di media sosial untuk menarik audiens yang lebih luas.")

    # Pendapatan tinggi
    if pendapatan >= 5000000:
        recommendations.append("Gunakan influencer untuk meningkatkan awareness produk.")
        recommendations.append("Adakan event besar atau promo besar-besaran untuk meningkatkan eksposur.")

    return recommendations

# Menambah lebih banyak contoh data sintesis dengan produk yang lebih umum dan satu kata
dataset_expanded_simple_v3 = []
for _ in range(1000):  # Generate 1000 baris data sintetik
    produk = random.choice(fashion_products_simple + fnb_products_simple)
    terjual = random.randint(10, 200)
    harga_satuan = random.randint(8000, 50000)
    pendapatan = terjual * harga_satuan
    # Keuntungan sebagai persentase dari pendapatan (misalnya 10% - 50%)
    keuntungan_percentage = random.uniform(0.1, 0.5)  # Persentase keuntungan antara 10% - 50%
    keuntungan = pendapatan * keuntungan_percentage

    prompt = (
        f"Produk: {produk}\n"
        f"Jumlah Terjual: {terjual}\n"
        f"Pendapatan: {pendapatan}\n"
        f"Keuntungan: {keuntungan}\n"
        f"Rekomendasi Promosi:"
    )

    # Generate rekomendasi berdasarkan persentase keuntungan dan pendapatan
    recommendations = generate_recommendations_v2(pendapatan, keuntungan)

    dataset_expanded_simple_v3.append({
        "produk": produk,
        "jumlah_terjual": terjual,
        "pendapatan": pendapatan,
        "keuntungan": keuntungan,
        "rekomendasi": " | ".join(recommendations)
    })

# Membuat DataFrame dan menyimpannya ke CSV
df_v3 = pd.DataFrame(dataset_expanded_simple_v3)

# Menyimpan ke file CSV
file_path_expanded_simple_v3_csv = "/content/expanded_synthetic_data_simple_v3.csv"
df_v3.to_csv(file_path_expanded_simple_v3_csv, index=False)

file_path_expanded_simple_v3_csv  # Mengembalikan path untuk diunduh

In [ ]:
# Membaca data dari CSV
df = pd.read_csv('/content/expanded_synthetic_data_simple_v3.csv')

# Preprocessing: Label encode produk agar bisa diproses oleh machine learning model
encoder = LabelEncoder()
df['produk_encoded'] = encoder.fit_transform(df['produk'])

# Daftar produk dari dataset
produk_list = df['produk'].unique()

# Preprocessing: Membuat representasi vektor menggunakan TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit dan transform produk dari dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(produk_list)

# Fungsi untuk mencari produk yang mirip berdasarkan input produk
def find_similar_product(input_product, tfidf_matrix, produk_list):
    # Transform input produk menjadi vektor TF-IDF
    input_vec = tfidf_vectorizer.transform([input_product])

    # Menghitung similarity dengan produk yang ada
    cosine_sim = cosine_similarity(input_vec, tfidf_matrix)

    # Mengambil indeks produk yang paling mirip
    similar_idx = cosine_sim.argsort()[0][-1]  # Ambil indeks produk dengan similarity tertinggi
    return produk_list[similar_idx]

# Menyiapkan data (Fitur dan Target)
features = ['produk_encoded', 'jumlah_terjual', 'pendapatan', 'keuntungan']
target = 'rekomendasi'

# Membagi dataset menjadi fitur (X) dan target (y)
X = df[features]
y = df[target]

# Membagi data menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Melatih model menggunakan RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Menyimpan model ke file .pkl
model_path = "/content/rekomendasi_model.pkl"
joblib.dump(model, model_path)

# Evaluasi Model
accuracy = accuracy_score(y_test, y_pred)
#print(f"Akurasi Model: {accuracy * 100:.2f}%")

# Fungsi untuk memprediksi rekomendasi berdasarkan input produk dan informasi
def predict_recommendation(input_product, jumlah_terjual, pendapatan, keuntungan):
    # Mencari produk yang mirip
    similar_product = find_similar_product(input_product, tfidf_matrix, produk_list)

    # Encode produk yang mirip
    similar_product_encoded = encoder.transform([similar_product])[0]

    # Buat input data untuk model
    input_data = pd.DataFrame({
        'produk_encoded': [similar_product_encoded],
        'jumlah_terjual': [jumlah_terjual],
        'pendapatan': [pendapatan],
        'keuntungan': [keuntungan]
    })

    # Prediksi rekomendasi dengan model
    recommendation = model.predict(input_data)
    return recommendation[0], similar_product

# Test prediksi untuk produk baru
input_product = "kaos"
jumlah_terjual = 120
pendapatan = 2400000
keuntungan = 500000

recommendation, similar_product = predict_recommendation(input_product, jumlah_terjual, pendapatan, keuntungan)
#print(f"Produk Mirip: {similar_product}")
print(f"Rekomendasi untuk '{input_product}': {recommendation}")

# Kembalikan path untuk diunduh
model_path

Akurasi Model: 93.00%
Produk Mirip: Kaos
Rekomendasi untuk 'kaos': Diskon moderat untuk meningkatkan volume penjualan. | Pertimbangkan untuk meningkatkan harga untuk meningkatkan margin. | Lakukan promosi di platform e-commerce dengan potongan harga. | Coba promosi di media sosial untuk menarik audiens yang lebih luas.


'/content/rekomendasi_model.pkl'

# TensorFlow Check

In [ ]:
import pandas as pd
import numpy as np
import random
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# ============================
# 1. Generate Synthetic Dataset
# ============================
fashion_products_simple = ["kaos", "kemeja", "celana", "jaket"]
fnb_products_simple = ["kopi", "teh", "nasi", "ayam"]

def generate_recommendations_v2(pendapatan, keuntungan):
    recommendations = []
    ratio = keuntungan / pendapatan if pendapatan != 0 else 0
    if ratio < 0.1:
        recommendations += ["Tawarkan diskon besar", "Strategi bundling", "Cari pemasok lebih murah"]
    elif 0.1 <= ratio < 0.3:
        recommendations += ["Diskon moderat", "Naikkan harga", "Promosi e-commerce"]
    else:
        recommendations += ["Pertahankan harga", "Investasi pemasaran", "Strategi premium"]

    if pendapatan < 2_000_000:
        recommendations.append("Diskon 10% untuk pelanggan baru")
    elif pendapatan < 5_000_000:
        recommendations.append("Promosi media sosial")
    else:
        recommendations += ["Gunakan influencer", "Promo besar-besaran"]

    return recommendations

# Generate data
dataset = []
for _ in range(1000):
    produk = random.choice(fashion_products_simple + fnb_products_simple)
    terjual = random.randint(10, 200)
    harga = random.randint(8000, 50000)
    pendapatan = terjual * harga
    keuntungan_pct = random.uniform(0.1, 0.5)
    keuntungan = pendapatan * keuntungan_pct
    rekom = generate_recommendations_v2(pendapatan, keuntungan)
    dataset.append({
        "produk": produk,
        "jumlah_terjual": terjual,
        "pendapatan": pendapatan,
        "keuntungan": keuntungan,
        "rekomendasi": " | ".join(rekom)
    })

# Save and load dataset
df = pd.DataFrame(dataset)
df.to_csv("synthetic_data.csv", index=False)
df = pd.read_csv("synthetic_data.csv")

# ============================
# 2. Preprocessing
# ============================
produk_encoder = LabelEncoder()
df['produk_encoded'] = produk_encoder.fit_transform(df['produk'])
produk_list = df['produk'].unique()
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(produk_list)

# ============================
# 3. Dataset for Model
# ============================
X = df[['produk_encoded', 'jumlah_terjual', 'pendapatan', 'keuntungan']]
y = df['rekomendasi']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# ============================
# 4. Build TensorFlow Model
# ============================
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# ============================
# 5. Save Model & Encoders
# ============================
model.save("rekomendasi_model_tf.h5")
joblib.dump(scaler, "rekomendasi_scaler.pkl")
joblib.dump(label_encoder, "rekomendasi_label_encoder.pkl")
joblib.dump(produk_encoder, "produk_encoder.pkl")

# ============================
# 6. Inferensi (Prediksi)
# ============================
def find_similar_product(input_product, tfidf_matrix, produk_list):
    input_vec = tfidf_vectorizer.transform([input_product])
    cosine_sim = cosine_similarity(input_vec, tfidf_matrix)
    similar_idx = cosine_sim.argsort()[0][-1]
    return produk_list[similar_idx]

def predict_recommendation(input_product, jumlah_terjual, pendapatan, keuntungan):
    similar_product = find_similar_product(input_product, tfidf_matrix, produk_list)
    similar_encoded = produk_encoder.transform([similar_product])[0]
    input_df = pd.DataFrame([{
        "produk_encoded": similar_encoded,
        "jumlah_terjual": jumlah_terjual,
        "pendapatan": pendapatan,
        "keuntungan": keuntungan
    }])
    input_scaled = scaler.transform(input_df)
    probs = model.predict(input_scaled)
    pred_class = np.argmax(probs, axis=1)
    pred_label = label_encoder.inverse_transform(pred_class)
    return pred_label[0], similar_product

# ============================
# 7. Contoh Penggunaan
# ============================
input_product = "kaos"
jumlah_terjual = 120
pendapatan = 2400000
keuntungan = 500000

hasil, mirip = predict_recommendation(input_product, jumlah_terjual, pendapatan, keuntungan)
print(f"Produk Mirip: {mirip}\nRekomendasi: {hasil}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 0.2665 - loss: 1.7419 - val_accuracy: 0.3313 - val_loss: 1.6497
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4628 - loss: 1.5887 - val_accuracy: 0.5188 - val_loss: 1.5125
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5535 - loss: 1.4425 - val_accuracy: 0.5625 - val_loss: 1.3524
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6078 - loss: 1.2723 - val_accuracy: 0.6375 - val_loss: 1.1997
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6378 - loss: 1.1264 - val_accuracy: 0.6062 - val_loss: 1.0722
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6613 - loss: 1.0211 - val_accuracy: 0.6750 - val_loss: 0.9620
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7360 - loss: 0.9201 - val_accuracy: 0.7312 - val_loss: 0.8625
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7441 - loss: 0.8282 - val_accuracy: 0.7375 - va

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
Produk Mirip: kaos
Rekomendasi: Diskon moderat | Naikkan harga | Promosi e-commerce | Promosi media sosial
